# Accessing TRACMIP data using `qgrid`

This is a sample of what data exploration would look like for TRACMIP using `qgrid`, which renders a dataframe in a form which can easily be filtered and sorted.

In [1]:
from intake import open_catalog
import pandas as pd
import qgrid

We open the `intake` master catalog and search for the TRACMIP entries:

In [2]:
cat = open_catalog("https://raw.githubusercontent.com/pangeo-bot/pangeo-datastore/master/master.yaml").search("tracmip")

With this catalog, we can now make a `pandas` dataframe and render it using the `qgrid` widget:

In [3]:
df = pd.DataFrame({"frequency"  : [item.split(".")[1] for item in list(cat)],
                   "experiment" : [item.split(".")[2] for item in list(cat)],
                   "model"      : [item.split(".")[3] for item in list(cat)],
                   "variable"   : [item.split(".")[4] for item in list(cat)],
                   "version"    : [item.split(".")[5] for item in list(cat)],
                   "source"     : [cat[item] for item in list(cat)]})

qg = qgrid.show_grid(df)
qg

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

From here, I'm trying to find a less sloppy way to pull the selected data from the widget to `xarray` datasets, but for now this gets all the desired datasets into one list:

In [4]:
ds_list = [item.to_dask() for item in qg.get_changed_df()["source"]]
ds_list

[<xarray.Dataset>
 Dimensions:    (bnds: 2, lat: 90, lon: 144, time: 480)
 Coordinates:
   * lat        (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
   * lon        (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
   * time       (time) object 0046-01-16 00:00:00 ... 0085-12-16 00:00:00
 Dimensions without coordinates: bnds
 Data variables:
     clivi      (time, lat, lon) float32 dask.array<shape=(480, 90, 144), chunksize=(480, 90, 144)>
     lat_bnds   (lat, bnds) float64 dask.array<shape=(90, 2), chunksize=(90, 2)>
     lon_bnds   (lon, bnds) float64 dask.array<shape=(144, 2), chunksize=(144, 2)>
     time_bnds  (time, bnds) object dask.array<shape=(480, 2), chunksize=(480, 2)>
 Attributes:
     Conventions:            CF-1.4
     branch_time:            0.0
     cmor_version:           2.9.1
     comment:                aqua planet with 4xco2 of TRACMIP; for TRACMIP se...
     contact:                Sarah Kang, skang.unist.ac.kr; Elizabeth Maroon, .